### Installation

In [ ]:
import os
import sys
import subprocess
from PIL import Image
import numpy as np

# Verify directory exists
THUMBNAILS_DIR = r"C:\Users\ChanK\OneDrive - Tilburg University\Thesis 2024\YT Thumbnails"
if not os.path.exists(THUMBNAILS_DIR):
    raise FileNotFoundError(f"Thumbnail directory not found: {THUMBNAILS_DIR}")

def install_packages():
    """Ensure all required packages are installed."""
    required_packages = [
        "deepface", 
        "opencv-python", 
        "torch", 
        "transformers", 
        "umap-learn", 
        "hdbscan", 
        "colorthief", 
        "easyocr", 
        "vaderSentiment", 
        "ultralytics"
    ]
    for package in required_packages:
        try:
            subprocess.run([sys.executable, "-m", "pip", "install", package], check=True)
        except subprocess.CalledProcessError as e:
            print(f"Failed to install {package}: {e}")
            sys.exit(1)

# Install all required packages before running the rest of the code
install_packages()

def get_thumbnails(num_samples=None):
    """Load first N thumbnails from directory."""
    files = [f for f in os.listdir(THUMBNAILS_DIR) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
    return [os.path.join(THUMBNAILS_DIR, f) for f in files[:num_samples]]

### Face Detection

In [ ]:
import cv2
import matplotlib.pyplot as plt
from deepface import DeepFace

def analyze_faces_deepface(num_samples=10):
    results = {}

    for img_path in get_thumbnails(num_samples):
        # Load image using OpenCV
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB for DeepFace

        try:
            # Analyze the image for face detection
            analysis = DeepFace.analyze(img_path, detector_backend="retinaface", actions=['age', 'gender', 'emotion'])

            face_count = len(analysis) if isinstance(analysis, list) else 1
            face_data = analysis if isinstance(analysis, list) else [analysis]

            # Draw bounding boxes on the image
            for face in face_data:
                if "region" in face:
                    x, y, w, h = face["region"]["x"], face["region"]["y"], face["region"]["w"], face["region"]["h"]
                    cv2.rectangle(img_rgb, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Green box

            results[img_path] = {
                'face_count': face_count,
                'face_data': face_data
            }

            # Display the image with face bounding boxes
            plt.figure(figsize=(6, 6))
            plt.imshow(img_rgb)
            plt.axis("off")
            plt.title(f"Detected Faces: {face_count}")
            plt.show()

        except Exception as e:
            results[img_path] = {'error': str(e)}
    
    return results


In [ ]:
analyze_faces_deepface(50)

### Composition

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

def analyze_composition(num_samples=10):
    comp_results = {}
    
    for img_path in get_thumbnails(num_samples):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB for correct colors

        # Compute saliency heatmap
        saliency = cv2.saliency.StaticSaliencyFineGrained_create()
        _, sal_map = saliency.computeSaliency(img)

        # Convert saliency map to a heatmap
        sal_map = (sal_map * 255).astype(np.uint8)  # Scale values to 0-255
        sal_map_colored = cv2.applyColorMap(sal_map, cv2.COLORMAP_JET)  # Apply heatmap
        sal_map_colored = cv2.cvtColor(sal_map_colored, cv2.COLOR_BGR2RGB)  # Convert for matplotlib

        # Rule of thirds grid calculation
        height, width = img.shape[:2]
        thirds = [(width//3, height//3), (2*width//3, 2*height//3)]

        comp_results[img_path] = {
            'saliency_heatmap': sal_map.tolist(),
            'rule_of_thirds_grid': thirds
        }

        # Display side-by-side images
        fig, axes = plt.subplots(1, 2, figsize=(10, 5))
        
        # Left: Original Image
        axes[0].imshow(img)
        axes[0].set_title("Original Image")
        axes[0].axis("off")

        # Right: Saliency Heatmap
        axes[1].imshow(sal_map_colored)
        axes[1].set_title("Saliency Heatmap")
        axes[1].axis("off")

        plt


In [ ]:
analyze_composition(50)

### Color Analysis

In [ ]:
import matplotlib.pyplot as plt
import cv2
from colorthief import ColorThief

def analyze_colors(num_samples=10):
    color_data = {}
    for img_path in get_thumbnails(num_samples):
        # Load and display the image
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB for correct colors

        plt.figure(figsize=(5, 5))
        plt.imshow(img)
        plt.axis('off')  # Hide axis
        plt.title(f"Image: {img_path.split('/')[-1]}")  # Show filename in title
        plt.show()

        # Perform color analysis
        ct = ColorThief(img_path)
        color_data[img_path] = {
            'dominant': ct.get_color(quality=1),
            'palette': ct.get_palette(color_count=5)
        }

        # Print the analysis
        print(f"  Dominant Color: {color_data[img_path]['dominant']}")
        print(f"  Color Palette: {color_data[img_path]['palette']}\n")

    return color_data


In [ ]:
analyze_colors(50)

### Object Detection

In [ ]:
import cv2
import torch
import matplotlib.pyplot as plt
from ultralytics import YOLO

def detect_objects(num_samples=10):
    model = YOLO('yolov8n.pt')  # Load YOLOv8 nano model
    results = {}

    for img_path in get_thumbnails(num_samples):
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for display

        # Run YOLOv8 object detection
        detections = model(img_path)[0]  # Get first result
        class_names = detections.names  # Dictionary of class ID to label
        
        objects_detected = []
        boxes = []

        for box, cls_id in zip(detections.boxes.xyxy.tolist(), detections.boxes.cls.tolist()):
            label = class_names[int(cls_id)]  # Convert class ID to name
            objects_detected.append(label)
            boxes.append(box)

            # Draw bounding box and label on image
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(img_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box
            cv2.putText(img_rgb, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        results[img_path] = {
            'objects': objects_detected,
            'boxes': boxes
        }

        # Show image with bounding boxes
        plt.figure(figsize=(6, 6))
        plt.imshow(img_rgb)
        plt.axis("off")
        plt.title(f"Detected Objects: {', '.join(objects_detected) if objects_detected else 'None'}")
        plt.show()

    return results


In [ ]:
detect_objects(50)

### OCR Detection

In [ ]:
import cv2
import matplotlib.pyplot as plt
from easyocr import Reader
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def analyze_text(num_samples=10):
    reader = Reader(['en'])
    sia = SentimentIntensityAnalyzer()
    
    text_results = {}
    
    for img_path in get_thumbnails(num_samples):
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert for matplotlib

        detections = reader.readtext(img_path)

        extracted_texts = []
        sentiment_scores = []
        
        for box, text, confidence in detections:
            extracted_texts.append(text)
            sentiment_scores.append({
                'text': text,
                'score': sia.polarity_scores(text)
            })

            # Draw bounding boxes around detected text
            x_min, y_min = map(int, box[0])  # Top-left corner
            x_max, y_max = map(int, box[2])  # Bottom-right corner
            cv2.rectangle(img_rgb, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            cv2.putText(img_rgb, text, (x_min, y_min - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        text_results[img_path] = {
            'text': extracted_texts,
            'sentiment': sentiment_scores
        }

        # Display the image with detected text boxes
        plt.figure(figsize=(6, 6))
        plt.imshow(img_rgb)
        plt.axis("off")
        plt.title("Detected Text & Sentiment")
        plt.show()
    
    return text_results


In [ ]:
analyze_text(50)

### Cluster Styles (CLIP)

In [ ]:
import torch
import numpy as np
import umap.umap_ as umap
import hdbscan
import matplotlib.pyplot as plt
import matplotlib.offsetbox as offsetbox
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

def cluster_styles(num_samples=10):
    # Load CLIP model and processor
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

    embeddings = []
    img_paths = get_thumbnails(num_samples)  # Get image paths

    for img_path in img_paths:
        image = Image.open(img_path)
        inputs = processor(images=image, return_tensors="pt")

        # Extract image embeddings
        with torch.no_grad():
            outputs = model.get_image_features(**inputs)

        embeddings.append(outputs.squeeze(0).detach().cpu().numpy())

    # Convert embeddings into NumPy array
    embeddings = np.vstack(embeddings)

    # Reduce dimensionality
    reducer = umap.UMAP(n_neighbors=5, min_dist=0.3, metric='cosine')
    reduced_embeds = reducer.fit_transform(embeddings)

    # Perform clustering
    clusterer = hdbscan.HDBSCAN(min_cluster_size=3, metric='euclidean')
    cluster_labels = clusterer.fit_predict(reduced_embeds)

    # Map images to clusters
    cluster_mapping = {img_path: label for img_path, label in zip(img_paths, cluster_labels)}

    # Visualization
    fig, ax = plt.subplots(figsize=(10, 6))
    scatter = ax.scatter(reduced_embeds[:, 0], reduced_embeds[:, 1], c=cluster_labels, cmap='tab10', s=50, edgecolors='k')

    # Add images as annotations
    for i, (x, y) in enumerate(reduced_embeds):
        img = Image.open(img_paths[i])
        img.thumbnail((30, 30))  # Resize for visualization
        imagebox = offsetbox.AnnotationBbox(offsetbox.OffsetImage(img, zoom=0.5), (x, y), frameon=False)
        ax.add_artist(imagebox)

    ax.set_title("Thumbnail Style Clustering (UMAP + HDBSCAN)")
    plt.colorbar(scatter, label="Cluster ID")
    plt.show()

    return cluster_mapping


In [ ]:
cluster_styles(400)

### Output